In [1]:
from optimizers import OptimizationTask, OptimizationResult, Optimizer, optimizers
from dataset import Dataset, Instance
from pennylane import QNode
import pennylane.numpy as np
from math import pi

In [2]:
def generate_tasks(
    circuit_id: str,
    num_qubits: int,
    num_layers: int,
    rp_params_shape: tuple,
    crp_params_shape: tuple,
    max_evaluations: int = 250,
    convergence_threshold: int = 1e-6,
    num_tasks: int = 100
) -> list[OptimizationTask]:
    return [
        OptimizationTask(
            circuit_id=circuit_id,
            num_qubits=num_qubits,
            num_layers=num_layers,
            initial_params=(
                np.random.random(rp_params_shape) * 2 * pi,
                np.random.random(crp_params_shape) * 4 * pi
            ),
            max_evaluations=max_evaluations,
            convergence_threshold=convergence_threshold
        )
        for _ in range(num_tasks)
    ]

In [3]:
def run_task(
        task: OptimizationTask,
        optimizers: list[tuple[str, Optimizer]]
) -> Instance:
    print(f"Optimizing an instance...")
    results: dict[str, OptimizationResult] = {
        name: optimize(task)
        for name, optimize in optimizers
    }
    print(f"Done optimizing an instance...")
    
    return Instance(
        task=task,
        results=results,
    )

In [4]:
from circuits import sim_03

num_qubits=4
num_layers=3
num_tasks = 2

tasks = generate_tasks(
    circuit_id="sim03",
    num_qubits=num_qubits,
    num_layers=num_layers,
    rp_params_shape = (num_layers, num_qubits, 2),
    crp_params_shape = (num_layers, num_qubits - 1),
    num_tasks=num_tasks,
    max_evaluations=125
)

In [5]:
from concurrent.futures import Future, ProcessPoolExecutor, as_completed
from loky.backend.context import cpu_count

cores = cpu_count()
print(f"Running with {cores} cores.")

folder = "../dataset/"

with ProcessPoolExecutor(max_workers=cores) as executor:
    task_map: dict[Future[Instance], OptimizationTask] = {
        executor.submit(run_task, task, optimizers): task
        for task in tasks
    }

    completed = 0
    for future in as_completed(task_map):
        instance: Instance = future.result()
        instance.save(folder)
        completed += 1
        print(f"Yeah I am done with a task look at me fellas {completed} / {num_tasks}")



Running with 8 cores.


Optimizing an instance...
Optimizing an instance...
yodl
yodl


/home/max/Projects/bachelor-thesis/code/minimization.py:22: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(reconstruction, x0, bounds=Bounds(0, 4 * math.pi), method='Nelder-Mead', tol=1e-6)
/home/max/Projects/bachelor-thesis/code/minimization.py:22: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(reconstruction, x0, bounds=Bounds(0, 4 * math.pi), method='Nelder-Mead', tol=1e-6)


yodl
yodl
yodl
yodl
yodl
yodl
yodl
yodl
yodl
yodl
0 0.4271809227584271
0 -0.6729313397041198
20 0.1925738772082798
20 -0.8206686156090046
40 -0.8969994329479589
40 -0.06139670045172252
60 -0.2965036078199443
60 -0.9355898806077132
0 0.4094464717658405
0 -0.6865448630005837
20 -0.958184592073448
20 -0.17879154810112385
40 -0.5757147092654086
40 -0.9879736664704292
60 -0.785778106443757
60 -0.9937210966303008
0 0.4094464686346269
0 -0.6865448556213962
20 0.20350884810942355
20 -0.8162847285693053
40 0.09092447152423633
40 -0.8650255729046714
60 0.004595012487061401
60 -0.8946781209741622
Done optimizing an instance...
Done optimizing an instance...
Yeah I am done with a task look at me fellas 1 / 2
Yeah I am done with a task look at me fellas 2 / 2
